This code is based on https://docs.dgl.ai/en/1.0.x/tutorials/blitz/3_message_passing.html#sphx-glr-tutorials-blitz-3-message-passing-py.

Other References:
https://docs.dgl.ai/en/1.0.x/guide/message-efficient.html#guide-message-passing-efficient

https://docs.dgl.ai/en/1.0.x/tutorials/models/1_gnn/9_gat.html#sphx-glr-tutorials-models-1-gnn-9-gat-py

In [1]:
! pip install  dgl -f https://data.dgl.ai/wheels/repo.html
! pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn

In [3]:
class GAT(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.lin1 = nn.Linear(in_features, out_features)
    self.a = nn.Linear(out_features *2, 1, bias = False)

  def update_e_ij(self, edges):
    # return edge features after update
    return {"e_ij": nn.LeakyReLU()(self.a(torch.cat([edges.src['z'], edges.dst['z']], dim=1)))}

  def reduce_func(self, nodes):
    # applies to the neighbors, return node features after update
    alpha_ij = nn.Softmax(dim =1)(nodes.mailbox["e_ij"])
    h = torch.sum(alpha_ij * nodes.mailbox["z"], dim = 1)
    return {"h": h}

  def message_func(self, edges):
    # return updated node and edge features
    return {"z": edges.src["z"], "e_ij": edges.data["e_ij"]}


  def forward(self, g, h):
    with g.local_scope(): # to prevent changes from going back to the original graph
      g.ndata["z"] = self.lin1(h)
      g.apply_edges(self.update_e_ij)

      g.update_all(self.message_func, self.reduce_func)

      return g.ndata["h"]



In [4]:
# For multi head attention
class MultiHeadAttentionGAT(nn.Module):
  def __init__(self, in_features, out_features, num_heads, agg="concat"):
    super().__init__()
    self.num_heads = num_heads
    self.agg = agg
    self.heads = nn.ModuleList()
    for i in range(self.num_heads):
      self.heads.append(GAT(in_features, out_features))

  def forward(self, g, h):
    res = []
    for i in range(self.num_heads):
      res.append(self.heads[i](g, h))
    if self.agg == "concat":
      return torch.cat(res, dim=1)
    else: # agg is mean
      return torch.mean(torch.stack(res))

In [5]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(30):
        # Forward
        logits = model(g, features)
        # print(logits)
        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


In [7]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        # self.conv1 = GAT(in_feats, h_feats)
        # self.conv2 = GAT(h_feats, num_classes)
        self.conv1 = MultiHeadAttentionGAT(in_feats, h_feats, 3, "concat")
        self.conv2 = MultiHeadAttentionGAT(h_feats*3, num_classes, 1, "concat")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.947, val acc: 0.058 (best 0.058), test acc: 0.064 (best 0.064)
In epoch 5, loss: 1.895, val acc: 0.338 (best 0.338), test acc: 0.361 (best 0.361)
In epoch 10, loss: 1.789, val acc: 0.692 (best 0.692), test acc: 0.666 (best 0.666)
In epoch 15, loss: 1.616, val acc: 0.726 (best 0.730), test acc: 0.691 (best 0.694)
In epoch 20, loss: 1.375, val acc: 0.752 (best 0.752), test acc: 0.711 (best 0.711)
In epoch 25, loss: 1.084, val acc: 0.768 (best 0.768), test acc: 0.752 (best 0.752)
